# SCENARI D'USO

In [28]:
import carla, time, sys,math, paho.mqtt.client as mqtt
from agents.navigation.basic_agent import BasicAgent
from enum import Enum
from Adas import Forward_collision_warning_mqtt

In [29]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

# FUNZIONI DI SUPPORTO

In [30]:
def load_map(world, map_name):
    if not world.get_map().name == f"Carla/Maps/{map_name}":
        world = client.load_world(map_name)

In [31]:
def destroy_all_vehicles(world):
    all_actors = world.get_actors()
    
    vehicles = all_actors.filter('vehicle.*')
    
    print(f"Numero di veicoli trovati: {len(vehicles)}")
    
    for vehicle in vehicles:
        vehicle.destroy()
        print(f"Veicolo distrutto: {vehicle.id}")

In [32]:
def destroy_all_pedestrians(world):
    all_actors = world.get_actors()
    
    pedestrians = all_actors.filter('walker.pedestrian.*')
    
    print(f"Numero di pedoni trovati: {len(pedestrians)}")
    
    for pedestrian in pedestrians:
        pedestrian.destroy()
        print(f"Pedone distrutto: {pedestrian.id}")

In [33]:
def spawn_vehicle(world, location, index, y = 0):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('vehicle.*')[index]
    spawn_point = carla.Transform()
    spawn_point.location = location
    spawn_point.rotation = carla.Rotation(yaw = y)
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

In [34]:
def actor_vehicle(world, vehicle, desitination, speed = 60):
    world.wait_for_tick()
    actor_agent = BasicAgent(vehicle)
    actor_agent.set_destination(desitination)
    actor_agent.set_target_speed(speed)
    actor_agent.ignore_vehicles()
    world.wait_for_tick()
    return actor_agent

In [35]:
def move_forward(vehicle, throttle):
    control = carla.VehicleControl()
    control.throttle = throttle
    vehicle.apply_control(control)

In [36]:
def move_agent(vehicle, actor_agent):
    vehicle.apply_control(actor_agent.run_step())

In [37]:
def move_forward_agent(vehicle, actor_agent,x,y):
    forward_location = vehicle.get_location() + carla.Location(x=x, y=y, z=0)  # Adjust 'x', 'y' based on vehicle's orientation
    actor_agent.set_destination(forward_location)
    vehicle.apply_control(actor_agent.run_step())

In [38]:
def stop_vehicle(vehicle):
    control = carla.VehicleControl()
    control.brake = 1.0 
    vehicle.apply_control(control)

In [39]:
# funzione inutile
def stop_vehicle_agent(vehicle):
    control = carla.VehicleControl()
    control.throttle = 0.0  # Nessuna accelerazione
    control.brake = 1.0     # Massima forza frenante
    control.hand_brake = True
    
    vehicle.apply_control(control)

In [40]:
def stop_simulation():
    global run
    run = False
    print("stop")

In [41]:
# Simulazione sensore umidità dell'asflato
def get_asphalt_friction_coefficient():
    world = client.get_world()
    weather = world.get_weather()
    weather.precipitation_deposits
    weather.wetness
    return 0.8

In [42]:
def disable_traffic_lights():
    traffic_lights = world.get_actors().filter('traffic.traffic_light')
    for traffic_light in traffic_lights:
        traffic_light.set_state(carla.TrafficLightState.Green)  # Imposta su verde
        traffic_light.set_green_time(99999)  # Mantiene il verde per un tempo molto lungo
        traffic_light.freeze(True)  # Disabilita la logica automatica
        print(f"Semaforo {traffic_light.id} impostato su verde permanente.")

In [43]:
def set_spectator_location(location, rotation):
    spectator = world.get_spectator()
    spectator.set_transform(carla.Transform(location, rotation))

In [44]:
def draw(world, r, g, b, l):
    debug = world.debug

    debug.draw_point(
        l,
        size=0.075,  # Dimensione del punto
        life_time=1200,  # Durata del punto
        persistent_lines=False,  # Il punto non persiste
        color=carla.Color(r, g, b)  # Colore blu
    )


## Camera

In [45]:
def view_from_above(vehicle):
    distance_above = 60.0  # Altezza della telecamera sopra il veicolo
    spectator = world.get_spectator()
    
    vehicle_location = vehicle.get_location()
    vehicle_rotation = vehicle.get_transform().rotation
    
    
    spectator_location = carla.Location(
            x=vehicle_location.x,  # Stessa posizione X del veicolo
            y=vehicle_location.y,  # Stessa posizione Y del veicolo
            z=vehicle_location.z + distance_above  # Posizionata più in alto rispetto al veicolo
        )

    spectator_rotation = carla.Rotation(
        pitch=-90.0,  # Guardare direttamente verso il basso
        yaw=0,  # Allineato con la rotazione del veicolo
        roll=0.0
    )
    
    spectator.set_transform(carla.Transform(spectator_location, spectator_rotation))
    

In [46]:
def view_from_behind(vehicle):
    distance_behind = 14.0  # How far behind the car the spectator will be
    height = 5.0  # Height of the camera (how high above the car it will be)
    distance_to_the_side = 0
    
    spectator = world.get_spectator()
    
    vehicle_location = vehicle.get_location()
    vehicle_rotation = vehicle.get_transform().rotation
    
    spectator_location = carla.Location(
            x=vehicle_location.x - distance_behind * vehicle_rotation.get_forward_vector().x + distance_to_the_side * vehicle_rotation.get_right_vector().x,
            y=vehicle_location.y - distance_behind * vehicle_rotation.get_forward_vector().y + distance_to_the_side * vehicle_rotation.get_right_vector().y,
            z=vehicle_location.z + height  # Keeping the camera above the vehicle
        )

        # Set the spectator's transform to follow the vehicle
   
    spectator.set_transform(carla.Transform(spectator_location, vehicle_rotation))
    

## ADAS

In [47]:
def attach_adas(world, vehicle):
    adas = Forward_collision_warning_mqtt(
        world = world,
        min_ttc = 0.9,
        attached_vehicle = vehicle,
        get_asphalt_friction_coefficient = lambda : get_asphalt_friction_coefficient(),
        action_listener = lambda : stop_simulation()
    )
    return adas

## Scenario 0: Viaggio senza ostacoli


Verificare che in condiziondi normali il veicolo viaggi tranquillamente

### setup

In [ ]:
disable_traffic_lights()

In [49]:
wps = [
    carla.Location(x=-41.5, y=117, z=1),
    carla.Location(x=-41.5, y=45, z=1),
    carla.Location(x=30, y=28.3, z=1)
]

### run

In [ ]:
destroy_all_vehicles(world)

vehicle = spawn_vehicle(world, wps[0], 0, -90)
world.wait_for_tick() # necessario per creare l'agente

actor_agent = actor_vehicle(world, vehicle, wps[1], 80)

wp = 1


try:
    run = True
    adas = attach_adas(world, vehicle)
    
    while run:
        view_from_behind(vehicle)
        
        if actor_agent.done():
            print("The target has been reached, searching for another target")
            wp = (wp +1) % len(wps)
            actor_agent.set_destination(wps[wp])
            
        vehicle.apply_control(actor_agent.run_step())
        #         world.wait_for_tick()
        time.sleep(0.02) # così lagga un po la cam
    stop_vehicle_agent(vehicle)
    
    time.sleep(2)
    
finally:
    print("done")
    adas.destroy()


## Scenario 1: Veicolo fermo o parcheggiato sulla carreggiata


Presenza di un veicolo parcheggiato direttamente davanti lungo la corsia di marcia.

In [ ]:
destroy_all_vehicles(world)

vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=117, z=1), 0, -90)
world.wait_for_tick()

destination = carla.Location(x=-41.5, y=45, z=1)

actor_agent = actor_vehicle(world, vehicle, destination, 120)


osbstacle_vehicle = spawn_vehicle(world, destination, 1, -90)

try:
    run = True
    adas = attach_adas(world, vehicle)
    
    while run:
#         view_from_above(vehicle)
        view_from_behind(vehicle)
        vehicle.apply_control(actor_agent.run_step())
        world.wait_for_tick()
        
    print(vehicle.get_location())
    draw(world, 0, 0, 255, vehicle.get_location())
    stop_vehicle_agent(vehicle)
    
    time.sleep(2)
    draw(world, 0, 0, 255, vehicle.get_location())
    
finally:
    print("done")
    adas.destroy()



Plot points

In [52]:
def draw(world, r, g, b, l):
    debug = world.debug

    debug.draw_point(
        l,
        size=0.075,  # Dimensione del punto
        life_time=1200,  # Durata del punto
        persistent_lines=False,  # Il punto non persiste
        color=carla.Color(r, g, b)  # Colore blu
    )


r, g, b = 0, 0, 255  # Colore blu

draw(world, 0, 0, 255, carla.Location(x=-41.628563, y=63.226833, z=-0.008968))
draw(world, 0, 0, 255, carla.Location(x=-41.659218, y=52.215012, z=-0.007616))


draw(world, 0, 255, 0, carla.Location(x=-41.595119, y=75.170609, z=-0.011463))
draw(world, 0, 255, 0, carla.Location(x=-41.626343, y=63.377842, z=-0.011196))



